In [6]:
!pip install surprise
import pandas as pd
import numpy as np

import surprise
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
rootdir = '/content/drive/My Drive/Colab Notebooks/Movie  Recommendation System/Movie Dataset/'

In [8]:
ratings = pd.read_csv(rootdir+'ratings.csv')

In [9]:
ratings.shape

(100836, 4)

In [10]:
movies = pd.read_csv(rootdir+'movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


**Data Preprocessing**

In [11]:
for dataset in [ratings,movies]:
  dataset.drop_duplicates(inplace=True)

In [12]:
ratings[ratings['userId'].isnull()]

,movieId,title,userId,rating


In [13]:
ratings[ratings['rating'].isnull()]

,movieId,title,userId,rating


**CF based recommendation system**

Our content based engine suffers from some severe limitations.

* It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.
* Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who (s)he is.
* Therefore, in this section, we will use Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.


**Item based**

In [14]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [15]:
userRatings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [16]:
userRatings = userRatings.replace(0.0,np.nan)

In [17]:
def normalize(row):
    new_row = (row - np.nanmean(row))
    return new_row

* since each user has its own standard we can't assume A's rating 7 is same as another users' 7. So we use Correlation similarity. We use Shifted Cosine similarity also known as Pearson correlation

In [18]:
shiftedratings = userRatings.apply(normalize,axis=1).T
shiftedratings = shiftedratings.replace(np.nan,0.0)
shiftedratings


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.609375,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.717268,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,-0.843202,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
(500) Days of Summer (2009),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.549618,0.0,0.0,0.285211,0.000000,0.000000,0.00000,-2.082609,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,-0.343202,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,-0.317408
10 Cloverfield Lane (2016),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.549618,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.182592
10 Things I Hate About You (1999),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.609375,0.0,0.0,0.000000,0.0,0.0,0.000000,0.282732,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,-1.65,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.65,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-1.02584,0.0,0.156798,0.000804,0.000000,0.0,-0.479574,0.0,1.727273,0.0,0.0,0.00000,0.0,0.000000
"10,000 BC (2008)",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,-2.527123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoolander (2001),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.714789,0.000000,-0.154589,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.774306,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,-0.866154,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,1.500804,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.18078,0.0,0.182592
Zootopia (2016),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.450382,0.0,0.0,0.000000,0.000000,0.000000,0.58477,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.651429,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.532738,0.00000,0.0,0.000000,0.000000,0.069149,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.182592
eXistenZ (1999),0.000000,0

In [19]:
cosine_sim = cosine_similarity(shiftedratings,shiftedratings)
cosine_sim

array([[ 1.        , -0.00996779,  0.        , ...,  0.02759571,
         0.05954319, -0.05959612],
       [-0.00996779,  1.        ,  0.06142722, ..., -0.13134147,
         0.01862581,  0.01107794],
       [ 0.        ,  0.06142722,  1.        , ...,  0.04256248,
         0.08872532,  0.01479151],
       ...,
       [ 0.02759571, -0.13134147,  0.04256248, ...,  1.        ,
         0.08068036, -0.12269709],
       [ 0.05954319,  0.01862581,  0.08872532, ...,  0.08068036,
         1.        ,  0.02297565],
       [-0.05959612,  0.01107794,  0.01479151, ..., -0.12269709,
         0.02297565,  1.        ]])

In [20]:
resultDf = pd.DataFrame(cosine_sim,index=shiftedratings.index,columns=shiftedratings.index)

In [21]:
resultDf

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,-0.009968,0.000000,-0.003022,0.040137,0.167164,-0.030304,-0.056416,-0.044050,-0.071806,-0.014233,0.215960,0.003305,0.012198,0.006214,-0.046956,0.134680,0.055939,0.085644,-0.111539,0.005519,0.040825,0.000000,-0.022313,0.015942,0.000000,-0.185694,0.000000,0.121142,-0.127685,0.000000,0.058140,0.003033,-0.080011,-0.045876,-0.084099,-0.020834,0.000000,0.003120,-0.000815,...,-0.013617,-0.012631,0.000000,-0.088305,0.013468,0.039625,0.056510,0.000000,-0.009484,0.039619,-0.067922,0.026323,-0.093493,-0.138400,0.042534,0.018276,-0.036760,0.017444,-0.121913,-0.167441,0.012643,0.039758,-0.143197,0.000000,-0.090475,-0.170854,-0.132040,0.134249,0.462067,0.104072,0.002768,-0.024753,-0.113362,-0.077760,0.001621,-0.018716,-0.018082,0.027596,0.059543,-0.059596
(500) Days of Summer (2009),-0.009968,1.000000,0.061427,0.019810,-0.079270,-0.031996,-0.016318,0.070459,0.144630,-0.039915,0.137074,-0.037391,-0.020713,0.132398,0.075082,-0.016229,-0.019167,-0.091431,-0.036076,0.016007,0.031447,-0.063668,-0.034735,0.002680,0.031802,-0.008458,-0.078684,-0.031059,-0.024731,0.131254,0.031257,0.036902,0.083136,0.000000,0.065889,-0.012268,-0.021802,0.003016,-0.170357,0.102290,...,-0.189299,0.012256,0.111728,0.023593,0.193283,0.024925,-0.007671,-0.006776,-0.051713,0.064536,0.014845,0.021214,-0.002034,0.026070,0.010710,0.019649,-0.003070,0.091216,0.000561,0.002759,-0.035092,0.078900,-0.065862,-0.041274,0.014497,0.046225,-0.024852,0.021747,-0.035994,0.027458,-0.074934,-0.039898,0.031983,0.239684,-0.016862,0.021079,-0.011978,-0.131341,0.018626,0.011078
10 Cloverfield Lane (2016),0.000000,0.061427,1.000000,-0.003553,-0.049735,-0.003786,-0.228422,0.079793,0.000000,0.043307,0.000000,0.000000,0.034710,0.116568,0.000000,-0.008080,0.000000,-0.038221,0.021771,-0.058119,0.000000,0.037794,-0.125818,0.000000,-0.169464,-0.037033,-0.056944,0.000000,0.102865,0.110575,0.073183,-0.028107,0.026865,0.000000,-

In [23]:
def get_similar_movies(title):
    if title not in resultDf.index:
        return None
    else:
        similarList = zip(resultDf[title],list(resultDf.columns.values))
        similarList = sorted(similarList, key=lambda x: x[0], reverse=True)
        return similarList[1:10]
get_similar_movies('Avengers, The (2012)')

[(0.39354047765290545, 'Guardians of the Galaxy (2014)'),
 (0.3350502012420595, 'Dark Knight, The (2008)'),
 (0.3315781528883245, 'Captain America: The Winter Soldier (2014)'),
 (0.3303951457597172, 'Iron Man (2008)'),
 (0.3208819343075565, 'How to Train Your Dragon (2010)'),
 (0.31121275341353893, 'Mission: Impossible - Rogue Nation (2015)'),
 (0.3046853872951562, 'Dark Knight Rises, The (2012)'),
 (0.3011995583901491, "World's End, The (2013)"),
 (0.29908709617117857, 'Harry Potter and the Order of the Phoenix (2007)')]